In [2]:
import pandas as pd
import numpy as np

import os
from pathlib import Path

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation, MaxPool2D
from keras.layers import Conv2D, MaxPooling2D

In [3]:
data_dir = Path('chest_xray')

In [4]:
train_dir=data_dir/'train'
test_dir=data_dir/'test'
val_dir=data_dir/'val'

In [5]:
file_path_train=list(train_dir.glob('*/*.jpeg'))
file_path_test=list(test_dir.glob('*/*.jpeg'))
file_path_val=list(val_dir.glob('*/*.jpeg'))

In [6]:
label_train=list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_path_train))

label_test=list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_path_test))

label_val=list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_path_val))

In [7]:
label_train=pd.Series(label_train,name='Label')

label_test=pd.Series(label_test,name='Label')

label_val=pd.Series(label_val,name='Label')

In [8]:
img_train=pd.Series(file_path_train,name='Filepath').astype(str)


img_test=pd.Series(file_path_test,name='Filepath').astype(str)


img_val=pd.Series(file_path_val,name='Filepath').astype(str)

In [9]:
train_df=pd.concat([img_train,label_train],axis=1)


test_df=pd.concat([img_test,label_test],axis=1)


val_df=pd.concat([img_val,label_val],axis=1)

In [10]:
l=LabelEncoder()
train_df1=pd.DataFrame()
test_df1=pd.DataFrame()
val_df1=pd.DataFrame()


train_df1["Label"]=l.fit_transform(train_df["Label"])
test_df1["Label"]=l.fit_transform(test_df["Label"])
val_df1["Label"]=l.fit_transform(val_df["Label"])

In [11]:
train_df1["Filepath"]=train_df["Filepath"]
test_df1["Filepath"]=test_df["Filepath"]
val_df1["Filepath"]=val_df["Filepath"]

In [12]:
train_gen = ImageDataGenerator(samplewise_center= True,
                                       samplewise_std_normalization= True)

In [13]:
train_image=train_gen.flow_from_directory(train_dir,
                                          batch_size = 32,
                                          target_size=(224,224))

Found 5216 images belonging to 2 classes.


In [14]:
#train_image_sample = ImageDataGenerator().flow_from_dataframe(dataframe=train_df, 
#                                                               x_col="Filepath", 
#                                                               y_col="Label", 
#                                                               class_mode="binary", 
#                                                               batch_size=100, 
#                                                               shuffle=True, 
#                                                               target_size=(224,224))

In [15]:
#batch= train_image_sample.next()
#data_sample= batch[0]

In [16]:
test_gen=ImageDataGenerator(featurewise_center=True,
                            featurewise_std_normalization= True)

In [17]:
#test_gen.fit(data_sample)

In [18]:
test_image=test_gen.flow_from_directory(test_dir,
                              batch_size=32,
                              target_size=(224,224))

Found 624 images belonging to 2 classes.


In [19]:
val_image=test_gen.flow_from_directory(val_dir,
                                       class_mode='binary',
                                       shuffle=False,
                                       target_size=(224,224))

Found 16 images belonging to 2 classes.


In [20]:
#def get_class_weights(labels):
#    weight_neg = np.sum(labels)/len(labels)
#    weight_pos= 1-weight_neg 
#    return weight_pos, weight_neg

In [21]:
#positive_weights, negative_weights= get_class_weights(train_image.labels)

In [22]:
#def get_weighted_loss_binary(pos_weights, neg_weights, epsilon=1e-7):
#    def weighted_loss(y_true, y_pred):
#        loss = 0.0
#        loss +=(-1*pos_weights*y_true* K.log(y_pred+epsilon)+ \
#            -1*neg_weights*(1-y_true)* K.log(1-y_pred+epsilon))
#        return loss
#    return weighted_loss#

In [23]:
#from keras.applications.densenet import DenseNet121

In [24]:
#base_model = DenseNet121(weights='DenseNet/DenseNet-BC-161-48-no-top.h5',include_top= False)
#x= base_model.output

#x= GlobalAveragePooling2D()(x)

#predictions= Dense(units= 1, activation ="sigmoid")(x)

#model= Model(inputs= base_model.input, outputs=predictions )

#model.compile(optimizer= 'adam', loss=get_weighted_loss_binary(positive_weights, negative_weights),
#             metrics= ['accuracy'])

In [25]:
#pip install keras.applications 

In [26]:
# number of filtered features- 32 kerenel size-(5,5)

#model1=models.Sequential()
#model1.add(Conv2D(32,(3,3),strides=(1, 1),activation='relu',padding='same', input_shape=(224, 224, 3)))
#model1.add(Dropout(0.3))
#model1.add(Conv2D(128,(5,5), strides=1,activation='relu',padding="same"))

#model1.add(Dense(1, activation='relu'))

In [27]:
#early_stopping_monitor = EarlyStopping(patience = 3, monitor = "val_acc", mode="max", verbose = 2)

In [28]:
#model1.compile(loss=get_weighted_loss_binary(positive_weights, negative_weights),
#             optimizer='adam',
#             metrics=['accuracy'])

In [29]:
#from keras import backend as K

In [30]:
#model1.fit_generator(
#           train_image,
#           epochs=5,
#           steps_per_epoch = len(train_image),
#           validation_data=test_image,
#           callbacks=[early_stopping_monitor]
#          )

In [31]:
model1 = Sequential()
model1.add(Conv2D(32,(3,3),strides=(1, 1),activation='relu',padding='same', input_shape=(224, 224, 3)))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Conv2D(64,(3,3),strides=(1, 1) ,padding='same',activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Conv2D(128,(3,3),strides=(1, 1),padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Conv2D(256,(3,3),strides=(1, 1),padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Flatten())

model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(2, activation='softmax'))

In [32]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [34]:


history = model1.fit_generator(train_image,epochs=10,validation_data=test_image,steps_per_epoch=10,callbacks=[early_stopping_cb])

/Users/lakshmipriya/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 0.9860 - accuracy: 0.6625

/Users/lakshmipriya/opt/anaconda3/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/lakshmipriya/opt/anaconda3/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


10/10 [==============================] - 63s 6s/step - loss: 0.9860 - accuracy: 0.6625 - val_loss: 24.2874 - val_accuracy: 0.6250
Epoch 2/10
10/10 [==============================] - 46s 5s/step - loss: 0.5230 - accuracy: 0.7437 - val_loss: 1.5972 - val_accuracy: 0.7821
Epoch 3/10
10/10 [==============================] - 47s 5s/step - loss: 0.2927 - accuracy: 0.8750 - val_loss: 65.0319 - val_accuracy: 0.6266
Epoch 4/10
10/10 [==============================] - 45s 5s/step - loss: 0.2652 - accuracy: 0.8813 - val_loss: 84.5604 - val_accuracy: 0.6250
Epoch 5/10
10/10 [==============================] - 39s 4s/step - loss: 0.2056 - accuracy: 0.9219 - val_loss: 79.6870 - val_accuracy: 0.6266
Epoch 6/10
10/10 [==============================] - 59s 6s/step - loss: 0.1227 - accuracy: 0.9469 - val_loss: 181.8852 - val_accuracy: 0.6250
Epoch 7/10
10/10 [==============================] - 64s 7s/step - loss: 0.1481 - accuracy: 0.9438 - val_loss: 196.1952 - val_accuracy: 0.6250


In [35]:
#model1.summary()

### model2

In [37]:
from keras.applications import VGG16

In [38]:
base_model=VGG16(include_top=False, weights=None,input_shape=(224,224,3), pooling='avg',classes=2)

In [39]:
base_model.load_weights("vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [40]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [41]:
model2=Sequential()
model2.add(base_model)
model2.add(Flatten())

model2.add(Dense(128,activation='relu'))
model2.add(Dense(64,activation='relu'))
model2.add(Dense(2,activation='softmax'))

## Freezing the layers
for layer in base_model.layers:
    layer.trainable=False

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
model2.fit_generator(train_image,epochs=20,validation_data=test_image,steps_per_epoch=50,callbacks=[early_stopping_cb])

Epoch 1/20
50/50 [==============================] - 800s 16s/step - loss: 0.4560 - accuracy: 0.7681 - val_loss: 0.8031 - val_accuracy: 0.8446
Epoch 2/20
50/50 [==============================] - 785s 16s/step - loss: 0.2036 - accuracy: 0.9262 - val_loss: 2.2446 - val_accuracy: 0.8397
Epoch 3/20
50/50 [==============================] - 810s 16s/step - loss: 0.1543 - accuracy: 0.9406 - val_loss: 3.2193 - val_accuracy: 0.8221
Epoch 4/20
50/50 [==============================] - 744s 15s/step - loss: 0.1300 - accuracy: 0.9525 - val_loss: 2.3686 - val_accuracy: 0.8702
Epoch 5/20
50/50 [==============================] - 800s 16s/step - loss: 0.1137 - accuracy: 0.9600 - val_loss: 2.6026 - val_accuracy: 0.8734
Epoch 6/20
50/50 [==============================] - 879s 18s/step - loss: 0.1183 - accuracy: 0.9506 - val_loss: 2.5307 - val_accuracy: 0.8798


In [43]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [44]:
# serialize model to JSON
model_json = model2.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [45]:
# serialize weights to HDF5
model2.save_weights("model2.h5")
print("Saved model to disk")

Saved model to disk
